In [1]:
import dart_fss as dart
import pandas as pd
import win32com.client
import time
import datetime

In [2]:
# DART API setting
api_key = 'ebef76b6fa887498f9588ceb2cea36512d042f87'
dart.set_api_key(api_key=api_key)

'ebef76b6fa887498f9588ceb2cea36512d042f87'

In [3]:
# 크레온 객체 생성
instCpCodeMgr = win32com.client.Dispatch("CpUtil.CpCodeMgr")

In [4]:
# 주식 codeList 반환
kospiList = instCpCodeMgr.GetStockListByMarket(1)
kosdacList = instCpCodeMgr.GetStockListByMarket(2)
codeList = kospiList + kosdacList
testList = codeList[0:10]

In [5]:
crp_list = dart.get_corp_list()

In [6]:
corp_codes = []
for i in range(len(testList)):
    code = testList[i][1:]
    main_code = crp_list.find_by_stock_code(code)
    if main_code != None:
        corp_codes.append(main_code)
    else:
        continue

Output()

Output()

In [7]:
main_codes = []
for i in range(len(corp_codes)):   
    main_codes.append(corp_codes[i].corp_code)

In [11]:
main_codes

['00119195',
 '00112378',
 '00101628',
 '00117744',
 '00126937',
 '00150244',
 '00145109']

In [171]:
code = '00101628 '

In [52]:
try:
    fs = dart.fs.extract(corp_code=code, bgn_de='20160101')
except RuntimeError as e:
    print(e)

In [183]:
fs = dart.fs.extract(corp_code=code, bgn_de='20160101')

Output()

Output()

In [184]:
#연결재무상태표 (자산, 부채, 자본)
df_fs = fs['bs']
labels_fs = fs.labels['bs']

# 연결포괄손익계산서 (수익, 이익, 손실)
df_ci = fs['cis']
labels_ci = fs.labels['cis']

In [185]:
df_fs.columns = df_fs.columns.droplevel()
df_ci.columns = df_ci.columns.droplevel()

df_fs = df_fs.iloc[:, 0:9]
df_ci = df_ci.iloc[:, 0:9]

fs_columns = ['concept_id', 'label_ko', 'label_en', 'class0', 'class1', 'class2', 'class3', 'fy2019', 'fy2018']
ci_columns = ['concept_id', 'label_ko', 'label_en', 'class0', 'class1', 'class2', 'class3', 'fy2019', 'fy2018']

df_fs.columns = fs_columns
df_ci.columns = ci_columns

df_fs = df_fs[['concept_id', 'label_ko', 'fy2019', 'fy2018']]
df_ci = df_ci[['concept_id', 'label_ko', 'fy2019', 'fy2018']]

In [186]:
gp_name = df_ci[df_ci['concept_id']=='ifrs-full_GrossProfit']['label_ko'].values[0]
gp = df_ci[df_ci['concept_id']=='ifrs-full_GrossProfit']['fy2019'].values[0]
a_name = df_fs[df_fs['concept_id']=='ifrs-full_Equity']['label_ko'].values[0]
a = df_fs[df_fs['concept_id']=='ifrs-full_Equity']['fy2019'].values[0]

In [187]:
gp/a

0.4045928551679975

In [189]:
df_ci

,concept_id,label_ko,fy2019,fy2018
0,ifrs-full_Revenue,수익(매출액),"307,150,025,786.0","306,602,589,029.0"
1,ifrs-full_CostOfSales,매출원가,"185,418,314,591.0","190,699,351,637.0"
2,ifrs-full_GrossProfit,매출총이익,"121,731,711,195.0","115,903,237,392.0"
3,dart_TotalSellingGeneralAdministrativeExpenses,판매비와관리비,"95,552,100,802.0","89,072,919,367.0"
4,entity00119195_udf_IS_2019324131845941_Stateme...,연구개발비,"16,261,864,137.0","15,604,537,990.0"
5,dart_OperatingIncomeLoss,영업이익(손실),"9,917,746,256.0","11,225,780,035.0"
6,dart_OtherGains,기타이익,"7,071,598,685.0","440,612,214.0"
7,dart_OtherLosses,기타손실,"1,522,684,917.0","241,131,399.0"
8,ifrs-full_FinanceIncome,금융수익,"2,509,166,305.0","3,034,177,786.0"
9,ifrs-full_FinanceCosts,금융원가,"956,030,319.0","240,914,422.0"


In [68]:
df_fs[df_fs['label_ko']=='자본총계']

,concept_id,label_ko,fy2019,fy2018
47,ifrs-full_Equity,자본총계,"2,029,507,000,000.0","2,001,942,000,000.0"


In [174]:
col1 = df_fs.columns.get_level_values(0)
col2 = df_fs.columns.get_level_values(1)

col1 = [col for col in col1 if col[:2] == '20']
col2 = [col for col in col2 if type(col) == str]

In [175]:
cols = col2+col1

In [176]:
cols

['concept_id',
 'label_ko',
 'label_en',
 'class0',
 'class1',
 'class2',
 'class3',
 '20191231',
 '20181231',
 '20171231']

In [177]:
df_fs.columns = df_fs.columns.droplevel()
df_fs.columns = cols

In [178]:
a = df_fs[df_fs['label_ko']=='자본총계']['20191231'].values[0]

In [179]:
a

300874594398.0

In [31]:
code = '00135917'

In [32]:
fs = dart.fs.extract(corp_code=code, bgn_de='20160101')

df_fs = fs['bs']     #연결재무상태표 (자산, 부채, 자본)
df_ci = fs['cis']    # 연결포괄손익계산서 (수익, 이익, 손실)

In [10]:
col1 = df_fs.columns.get_level_values(0)
col2 = df_fs.columns.get_level_values(1)
col3 = df_ci.columns.get_level_values(0)
col4 = df_ci.columns.get_level_values(1)


col1 = [col for col in col1 if col[:2] == '20']
col2 = [col for col in col2 if type(col) == str]
col3 = [col for col in col3 if col[:2] == '20']
col4 = [col for col in col4 if type(col) == str]
cols_fs = col2+col1
cols_ci = col4+col3

df_fs.columns = df_fs.columns.droplevel()
df_ci.columns = df_ci.columns.droplevel()

df_fs.columns = cols_fs
df_ci.columns = cols_ci

gp = df_ci[df_ci['label_ko']=='매출총이익']['20190101-20191231'].values[0]
a = df_fs[df_fs['label_ko']=='자본총계']['20191231'].values[0]

print('{} code score : {}'.format(code, gp/a))

IndexError: index 0 is out of bounds for axis 0 with size 0

In [33]:
df_ci

Statement of comprehensive income(Unit: KRW)             20190101-20191231  \
                                       label_ko   comment           (연결재무제표,)   
0                                        I.영업수익           8,027,630,868,453.0   
1                                       1.보험료수익     29,40 5,981,027,483,251.0   
2                                       2.재보험수익        28 1,284,212,880,093.0   
3                                        3.구상이익        14     3,494,540,754.0   
4                                        4.이자수익     28,40   354,335,883,747.0   
5                                 5.유가증권평가및처분이익        28    47,431,923,393.0   
6                                 6.대출채권평가및처분이익      9,28     1,401,069,162.0   
7                                   7.외환거래및환산이익        28    83,473,506,971.0   
8                               8.파생금융상품평가및처분이익     13,28    47,600,419,456.0   
9                                        9.기타수익  28,31,40   224,509,768,281.0   
10                                 10.특별계정수입수수료                  49,455,965.0   
11                                    11.특별계정수익        17        93,937,380.0   
12                                      II.영업비용           8,121,719,316,901.0   
13                                  1.보험계약부채전입액           1,233,799,488,532.0   
14                                      2.지급보험금        32 3,971,269,835,583.0   
15                                      3.재보험비용        30 1,402,712,746,701.0   
16                                        4.사업비     33,40   692,043,609,688.0   
17                                  5.이연신계약비상각비        14   557,861,659,765.0   
18                                      6.재산관리비     34,40    13,416,162,167.0   
19                                7.유가증권평가및처분손실        28    49,323,889,400.0   
20                                8.대출채권평가및처분손실      9,28    11,318,038,863.0   
21                                  9.외환거래및환산손실        28    11,009,473,176.0   
22                             10.파생금융상품평가및처분손실     13,28   130,427,573,465.0   
23                                      11.이자비용        28    25,274,320,156.0   
24                                      12.기타비용     35,40    23,160,517,011.0   
25                                 13.특별계정지급수수료                   8,065,014.0   
26                                    14.특별계정비용        17        93,937,380.0   
27                                 III.영업이익(손실)             -94,088,448,448.0   
28                                     IV.영업외수익        36     1,693,972,586.0   
29                                      V.영업외비용        37     2,543,159,206.0   
30                           VI.법인세비용차감전순이익(손실)             -94,937,635,068.0   
31                                    VII.법인세비용        38   -25,863,785,514.0   
32                               VIII.당기순이익(손실)             -69,073,849,554.0   
33                                    IX.기타포괄손익              97,819,599,296.0   
34                       1.후속적으로당기손익으로재분류될수없는항목              -7,198,325,873.0   
35                              1)순확정급여부채의재측정요소              -7,198,325,873.0   
36                       2.후속적으로당기손익으로재분류될수있는항목             105,017,925,169.0   
37                               1)매도가능금융자산평가손익             118,934,366,526.0   
38                               2)만기보유금융자산평가손익              -3,772,667,490.0   
39                                   3)파생상품평가손익             -10,144,161,542.0   
40                                 4)특별계정기타포괄손익                     387,675.0   
41                                    X.당기총포괄이익              28,745,749,742.0   
42                              XI.당기순이익(손실)의귀속             -69,073,849,554.0   
43                                     지배기업의소유주             -66,196,743,735.0   
44                                        비지배지분              -2,877,105,819.0   
45                               XII.당기총포괄이익의귀속              28,745,749,742.0   
46                                     지배기업의소유주              31,622,855,561.0   
47                                